# SRCNN

In [1]:
import torch
#import torch.nn as nn
#import torch.optim as optim

In [2]:
import h5py
import numpy as np
from torch.utils.data import Dataset

class TrainDataset(Dataset):
    def __init__(self, h5_file):
        super(TrainDataset, self).__init__()
        self.h5_file = h5_file

    def __getitem__(self, idx):
        with h5py.File(self.h5_file, 'r') as f:
            return np.expand_dims(f['lr'][idx] / 255., 0), np.expand_dims(f['hr'][idx] / 255., 0)

    def __len__(self):
        with h5py.File(self.h5_file, 'r') as f:
            return len(f['lr'])

In [3]:
dataset = TrainDataset('data/91-image_x4.h5')
len(dataset)

21772

In [4]:
dataset[0][1] * 255

array([[[ 90.007645,  86.78386 ,  88.59156 , ..., 113.089905,
         107.00605 , 106.247406],
        [ 87.44295 ,  84.47039 ,  88.59156 , ..., 112.48623 ,
         103.935585, 103.73811 ],
        [ 89.191925,  94.32131 ,  91.09919 , ..., 103.93392 ,
         100.514336, 104.18516 ],
        ...,
        [109.06166 ,  92.56571 ,  90.19351 , ..., 125.60508 ,
         128.26933 , 130.42783 ],
        [ 97.55636 ,  80.048874,  85.06247 , ..., 125.95421 ,
         133.39706 , 134.15572 ],
        [ 86.093544,  92.019104,  92.75821 , ..., 131.88309 ,
         140.13835 , 136.46753 ]]], shape=(1, 33, 33), dtype=float32)

In [5]:
def convert_ycbcr_to_rgb(img):
    if type(img) == np.ndarray:
        r = 298.082 * img[:, :, 0] / 256. + 408.583 * img[:, :, 2] / 256. - 222.921
        g = 298.082 * img[:, :, 0] / 256. - 100.291 * img[:, :, 1] / 256. - 208.120 * img[:, :, 2] / 256. + 135.576
        b = 298.082 * img[:, :, 0] / 256. + 516.412 * img[:, :, 1] / 256. - 276.836
        return np.array([r, g, b]).transpose([1, 2, 0])
    elif type(img) == torch.Tensor:
        if len(img.shape) == 4:
            img = img.squeeze(0)
        r = 298.082 * img[0, :, :] / 256. + 408.583 * img[2, :, :] / 256. - 222.921
        g = 298.082 * img[0, :, :] / 256. - 100.291 * img[1, :, :] / 256. - 208.120 * img[2, :, :] / 256. + 135.576
        b = 298.082 * img[0, :, :] / 256. + 516.412 * img[1, :, :] / 256. - 276.836
        return torch.cat([r, g, b], 0).permute(1, 2, 0)
    else:
        raise Exception('Unknown Type', type(img))

In [12]:
convert_ycbcr_to_rgb(dataset[0][1]).shape

(1, 33, 3)

In [10]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import io

# Load and display the image
#img = mpimg.imread("path_to_image.jpg")  # Replace with your image path
# Convert bytes to a BytesIO stream
image_stream = io.BytesIO(convert_ycbcr_to_rgb(dataset[0][1]))

# Open the image using PIL
image = Image.open(image_stream)
plt.imshow(image)
plt.axis("off")  # Hide axes
plt.show()


ValueError: ndarray is not C-contiguous

In [13]:
import tqdm